In [16]:
# Environment Setup for Production Inference
import sys
import os
import json
import datetime
import time
from typing import Dict, List, Any, Optional, Tuple

# Fix path for snowflake_connection module
current_dir = os.getcwd()
if "notebooks" in current_dir:
    src_path = os.path.join(current_dir, "..", "src")
else:
    src_path = os.path.join(current_dir, "src")

sys.path.append(src_path)
print(f"📁 Added to Python path: {src_path}")

from snowflake_connection import get_session
from snowflake.snowpark.functions import (
    col, lit, when, count, avg, sum as sum_, max as max_, min as min_,
    current_timestamp, call_udf, sql_expr, udf
)
from snowflake.snowpark.types import (
    StructType, StructField, StringType, DoubleType, IntegerType,
    FloatType, BooleanType, TimestampType
)

# Get Snowflake session
session = get_session()
print("✅ Environment ready for production inference")
print("🔮 Capabilities: Real-time UDFs, Batch Processing, Streamlit Integration")
print("⚡ Tools: Production inference, monitoring, external API integration")


📁 Added to Python path: /Users/beddy/Desktop/Github/Snowflake_ML_HCLS/notebooks/../src
🔄 Reusing existing Snowflake session
✅ Environment ready for production inference
🔮 Capabilities: Real-time UDFs, Batch Processing, Streamlit Integration
⚡ Tools: Production inference, monitoring, external API integration


In [17]:
# Stage and UDF Setup for Inference - MANDATORY UDF CREATION
print("🔧 Setting up ML infrastructure...")
print("⚠️ UDF creation is REQUIRED - will not proceed without it")

# Step 1: Create stage using multiple approaches until one works
print("📁 Creating ML models stage...")
stage_created = False
stage_name = None

# Approach 1: Try with full schema qualification
try:
    session.sql("""
        CREATE STAGE IF NOT EXISTS ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.ML_MODELS_STAGE
        COMMENT = 'Stage for storing ML model artifacts and UDF dependencies'
    """).collect()
    stage_name = "ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.ML_MODELS_STAGE"
    stage_created = True
    print("✅ ML models stage created with full schema qualification")
except Exception as e:
    print(f"⚠️ Full schema stage creation failed: {e}")

# Approach 2: Try with current schema only
if not stage_created:
    try:
        session.sql("CREATE STAGE IF NOT EXISTS ML_MODELS_STAGE").collect()
        stage_name = "ML_MODELS_STAGE"
        stage_created = True
        print("✅ ML models stage created in current schema")
    except Exception as e:
        print(f"⚠️ Current schema stage creation failed: {e}")

# Approach 3: Try with default temp stage
if not stage_created:
    try:
        session.sql("CREATE STAGE IF NOT EXISTS TEMP_ML_STAGE").collect()
        stage_name = "TEMP_ML_STAGE"
        stage_created = True
        print("✅ Temporary ML stage created")
    except Exception as e:
        print(f"⚠️ Temp stage creation failed: {e}")

if not stage_created:
    print("❌ CRITICAL: Could not create any stage")
    print("💡 Trying UDF creation without stage location...")
    stage_name = None

# Step 2: Create UDF with stage if available, without stage if necessary
print("🔧 Creating healthcare risk scoring UDF...")

# Try with stage first if available
udf_created = False
if stage_name:
    try:
        @udf(name="healthcare_risk_score_udf", 
             input_types=[FloatType(), IntegerType(), IntegerType(), IntegerType()],
             return_type=FloatType(),
             replace=True,
             stage_location=f"@{stage_name}")
        def healthcare_risk_score_with_stage(age: float, conditions: int, medications: int, claims: int) -> float:
            """Healthcare risk scoring UDF with stage location"""
            base_risk = (age / 100.0) * 25
            condition_risk = conditions * 6
            medication_risk = medications * 3
            utilization_risk = (claims / 10.0) * 4
            total_risk = base_risk + condition_risk + medication_risk + utilization_risk
            return min(100.0, max(0.0, total_risk))
        
        udf_created = True
        print(f"✅ UDF created WITH stage location: @{stage_name}")
        
    except Exception as e:
        print(f"⚠️ UDF creation with stage failed: {e}")
        print("🔄 Trying without stage location...")

# Try without stage if stage approach failed
if not udf_created:
    try:
        @udf(name="healthcare_risk_score_udf", 
             input_types=[FloatType(), IntegerType(), IntegerType(), IntegerType()],
             return_type=FloatType(),
             replace=True)
        def healthcare_risk_score_no_stage(age: float, conditions: int, medications: int, claims: int) -> float:
            """Healthcare risk scoring UDF without stage location"""
            base_risk = (age / 100.0) * 25
            condition_risk = conditions * 6
            medication_risk = medications * 3
            utilization_risk = (claims / 10.0) * 4
            total_risk = base_risk + condition_risk + medication_risk + utilization_risk
            return min(100.0, max(0.0, total_risk))
        
        udf_created = True
        print("✅ UDF created WITHOUT stage location")
        
    except Exception as e:
        print(f"❌ CRITICAL ERROR: UDF creation failed completely: {e}")
        print("🚨 Cannot proceed without UDF - check permissions and database setup")
        raise Exception(f"UDF creation is mandatory but failed: {e}")

# Step 3: Test the UDF - MANDATORY
print("🧪 Testing UDF creation...")
try:
    test_result = session.sql("""
        SELECT healthcare_risk_score_udf(65.0, 5, 8, 25) as RISK_SCORE
    """).collect()
    
    risk_score = test_result[0]['RISK_SCORE']
    print(f"✅ UDF test SUCCESSFUL - Test risk score: {risk_score:.2f}")
    print("🎯 UDF is working and ready for production inference!")
    udf_available = True
    
except Exception as e:
    print(f"❌ CRITICAL ERROR: UDF test failed: {e}")
    print("🚨 UDF exists but is not functional")
    raise Exception(f"UDF test is mandatory but failed: {e}")

print("✅ ALL CHECKS PASSED - UDF is fully operational")


🔧 Setting up ML infrastructure...
⚠️ UDF creation is REQUIRED - will not proceed without it
📁 Creating ML models stage...
✅ ML models stage created with full schema qualification
🔧 Creating healthcare risk scoring UDF...
✅ UDF created WITH stage location: @ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.ML_MODELS_STAGE
🧪 Testing UDF creation...
✅ UDF test SUCCESSFUL - Test risk score: 80.25
🎯 UDF is working and ready for production inference!
✅ ALL CHECKS PASSED - UDF is fully operational


In [18]:
# Real-time Inference Pipeline Setup - UDF REQUIRED
print("⚡ Setting up real-time inference pipeline...")
print("✅ UDF is confirmed operational - proceeding with UDF-only inference")

# Create real-time inference wrapper function
def predict_patient_risk(patient_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Real-time patient risk prediction using UDF (REQUIRED)
    """
    start_time = time.time()
    
    try:
        # Extract patient features
        age = float(patient_data.get('age', 0))
        conditions = int(patient_data.get('num_conditions', 0))
        medications = int(patient_data.get('num_medications', 0))
        claims = int(patient_data.get('num_claims', 0))
        
        # Make prediction using UDF (MANDATORY - no fallback)
        prediction_sql = f"""
            SELECT 
                healthcare_risk_score_udf({age}, {conditions}, {medications}, {claims}) as RISK_SCORE,
                CASE 
                    WHEN healthcare_risk_score_udf({age}, {conditions}, {medications}, {claims}) < 30 THEN 'LOW'
                    WHEN healthcare_risk_score_udf({age}, {conditions}, {medications}, {claims}) < 70 THEN 'MEDIUM'
                    ELSE 'HIGH'
                END as RISK_CATEGORY
        """
        
        result = session.sql(prediction_sql).collect()[0]
        risk_score = float(result['RISK_SCORE'])
        risk_category = result['RISK_CATEGORY']
        
        # Calculate prediction metadata
        prediction_time = (time.time() - start_time) * 1000  # Convert to milliseconds
        
        # Prepare comprehensive response
        response = {
            'patient_id': patient_data.get('patient_id', 'UNKNOWN'),
            'risk_score': risk_score,
            'risk_category': risk_category,
            'prediction_timestamp': datetime.datetime.now().isoformat(),
            'prediction_time_ms': round(prediction_time, 2),
            'model_version': 'v1.0.0',
            'confidence': 0.85,  # Simulated confidence score
            'input_features': {
                'age': age,
                'num_conditions': conditions,
                'num_medications': medications,
                'num_claims': claims
            },
            'clinical_recommendations': generate_clinical_recommendations(risk_score, risk_category),
            'success': True,
            'inference_method': 'UDF'  # Always UDF
        }
        
        # Log inference request
        log_inference_request(response)
        
        return response
        
    except Exception as e:
        # If UDF fails, this is a critical error since UDF is mandatory
        print(f"❌ CRITICAL: UDF inference failed: {e}")
        error_response = {
            'patient_id': patient_data.get('patient_id', 'UNKNOWN'),
            'error': f"UDF_FAILURE: {str(e)}",
            'prediction_timestamp': datetime.datetime.now().isoformat(),
            'prediction_time_ms': (time.time() - start_time) * 1000,
            'success': False
        }
        
        return error_response

def generate_clinical_recommendations(risk_score: float, risk_category: str) -> List[str]:
    """Generate clinical recommendations based on risk score"""
    
    recommendations = []
    
    if risk_category == 'HIGH':
        recommendations.extend([
            "🚨 High risk patient - Consider immediate clinical review",
            "📋 Review medication interactions and dosages", 
            "🩺 Schedule follow-up within 2 weeks",
            "📊 Monitor vital signs and laboratory values closely"
        ])
    elif risk_category == 'MEDIUM':
        recommendations.extend([
            "⚠️ Moderate risk - Schedule routine follow-up",
            "💊 Review medication adherence",
            "📅 Consider preventive care measures",
            "📈 Monitor for symptom progression"
        ])
    else:  # LOW
        recommendations.extend([
            "✅ Low risk - Continue routine care",
            "🏃 Encourage healthy lifestyle maintenance",
            "📅 Schedule annual wellness check",
            "📚 Provide patient education resources"
        ])
    
    # Add specific recommendations based on risk score
    if risk_score > 80:
        recommendations.append("🏥 Consider hospitalization or intensive monitoring")
    elif risk_score > 60:
        recommendations.append("🔄 Increase monitoring frequency")
    
    return recommendations

def log_inference_request(response: Dict[str, Any]):
    """Log inference request for monitoring and analysis"""
    
    try:
        log_data = [(
            f"REQ_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_{response['patient_id']}",
            'healthcare_risk_model',
            datetime.datetime.now().isoformat(),
            response['prediction_time_ms'],
            json.dumps(response['input_features']),
            response.get('risk_score', 0.0),
            'INFERENCE_PIPELINE',
            response['success']
        )]
        
        log_schema = StructType([
            StructField("REQUEST_ID", StringType()),
            StructField("MODEL_NAME", StringType()),
            StructField("REQUEST_TIMESTAMP", StringType()),
            StructField("RESPONSE_TIME_MS", DoubleType()),
            StructField("INPUT_FEATURES", StringType()),
            StructField("PREDICTION_RESULT", DoubleType()),
            StructField("REQUEST_SOURCE", StringType()),
            StructField("SUCCESS_STATUS", BooleanType())
        ])
        
        log_df = session.create_dataframe(log_data, schema=log_schema)
        log_df.write.mode("append").save_as_table("ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.INFERENCE_REQUEST_LOG")
        
    except Exception as e:
        print(f"⚠️ Logging error: {e}")

# Test real-time inference with UDF
print("🧪 Testing UDF-based inference pipeline...")

test_patients = [
    {'patient_id': 'TEST_001', 'age': 65, 'num_conditions': 5, 'num_medications': 8, 'num_claims': 25},
    {'patient_id': 'TEST_002', 'age': 35, 'num_conditions': 2, 'num_medications': 1, 'num_claims': 5},
    {'patient_id': 'TEST_003', 'age': 78, 'num_conditions': 12, 'num_medications': 15, 'num_claims': 45}
]

for patient in test_patients:
    result = predict_patient_risk(patient)
    if result['success']:
        print(f"   Patient {result['patient_id']}: {result['risk_score']:.1f} ({result['risk_category']}) - {result['prediction_time_ms']}ms [UDF]")
    else:
        print(f"   Patient {patient['patient_id']}: ❌ {result.get('error', 'Unknown error')}")

print("✅ UDF-based inference pipeline is operational")


⚡ Setting up real-time inference pipeline...
✅ UDF is confirmed operational - proceeding with UDF-only inference
🧪 Testing UDF-based inference pipeline...
   Patient TEST_001: 80.2 (HIGH) - 1385.1ms [UDF]
   Patient TEST_002: 25.8 (LOW) - 1152.04ms [UDF]
   Patient TEST_003: 100.0 (HIGH) - 1081.1ms [UDF]
✅ UDF-based inference pipeline is operational


In [19]:
# Streamlit Healthcare Dashboard Application
print("🖥️ Creating Streamlit healthcare dashboard...")

# Note: Streamlit imports are in the external file, not in this notebook

# Create the Streamlit application code
streamlit_app_code = '''
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import datetime
import time
import sys
import os

# Fix path for snowflake_connection module
current_dir = os.getcwd()
if "notebooks" in current_dir:
    src_path = os.path.join(current_dir, "..", "src")
else:
    src_path = os.path.join(current_dir, "src")

sys.path.append(src_path)

from snowflake_connection import get_session
from snowflake.snowpark.functions import col, lit, when, count, avg, sum as sum_, max as max_, min as min_

# Initialize Snowflake session
@st.cache_resource
def get_snowflake_session():
    return get_session()

session = get_snowflake_session()

# Page configuration
st.set_page_config(
    page_title="🏥 Healthcare Risk Assessment Dashboard",
    page_icon="🏥",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Dashboard title and header
st.title("🏥 Healthcare Risk Assessment Dashboard")
st.markdown("### Real-time Patient Risk Scoring & Clinical Decision Support")

# Sidebar for patient input
st.sidebar.header("🩺 Patient Risk Assessment")

# Patient input form
with st.sidebar.form("patient_form"):
    st.subheader("Enter Patient Information")
    
    patient_id = st.text_input("Patient ID", value="PAT_001")
    age = st.slider("Age", 0, 120, 65)
    num_conditions = st.slider("Number of Conditions", 0, 20, 5)
    num_medications = st.slider("Number of Medications", 0, 30, 8)
    num_claims = st.slider("Number of Claims (last year)", 0, 100, 25)
    
    submitted = st.form_submit_button("🔍 Calculate Risk Score")

# Main dashboard content
if submitted:
    # Calculate risk using UDF
    with st.spinner("Calculating risk score..."):
        try:
            # Call the healthcare risk UDF
            prediction_sql = f"""
                SELECT 
                    healthcare_risk_score_udf({age}, {num_conditions}, {num_medications}, {num_claims}) as RISK_SCORE,
                    CASE 
                        WHEN healthcare_risk_score_udf({age}, {num_conditions}, {num_medications}, {num_claims}) < 30 THEN 'LOW'
                        WHEN healthcare_risk_score_udf({age}, {num_conditions}, {num_medications}, {num_claims}) < 70 THEN 'MEDIUM'
                        ELSE 'HIGH'
                    END as RISK_CATEGORY
            """
            
            result = session.sql(prediction_sql).collect()[0]
            risk_score = float(result['RISK_SCORE'])
            risk_category = result['RISK_CATEGORY']
            
            # Display results
            col1, col2, col3 = st.columns(3)
            
            with col1:
                if risk_category == 'HIGH':
                    st.error(f"🚨 **HIGH RISK**")
                elif risk_category == 'MEDIUM':
                    st.warning(f"⚠️ **MEDIUM RISK**")
                else:
                    st.success(f"✅ **LOW RISK**")
                
                st.metric("Risk Score", f"{risk_score:.1f}")
            
            with col2:
                st.metric("Patient ID", patient_id)
                st.metric("Age", f"{age} years")
            
            with col3:
                st.metric("Conditions", num_conditions)
                st.metric("Medications", num_medications)
                st.metric("Claims", num_claims)
            
            # Risk gauge chart
            fig_gauge = go.Figure(go.Indicator(
                mode = "gauge+number+delta",
                value = risk_score,
                domain = {'x': [0, 1], 'y': [0, 1]},
                title = {'text': "Healthcare Risk Score"},
                delta = {'reference': 50},
                gauge = {
                    'axis': {'range': [None, 100]},
                    'bar': {'color': "darkblue"},
                    'steps': [
                        {'range': [0, 30], 'color': "lightgreen"},
                        {'range': [30, 70], 'color': "yellow"},
                        {'range': [70, 100], 'color': "red"}
                    ],
                    'threshold': {
                        'line': {'color': "red", 'width': 4},
                        'thickness': 0.75,
                        'value': 90
                    }
                }
            ))
            
            fig_gauge.update_layout(height=400)
            st.plotly_chart(fig_gauge, use_container_width=True)
            
            # Clinical recommendations
            st.subheader("📋 Clinical Recommendations")
            
            if risk_category == 'HIGH':
                recommendations = [
                    "🚨 High risk patient - Consider immediate clinical review",
                    "📋 Review medication interactions and dosages",
                    "🩺 Schedule follow-up within 2 weeks",
                    "📊 Monitor vital signs and laboratory values closely"
                ]
                if risk_score > 80:
                    recommendations.append("🏥 Consider hospitalization or intensive monitoring")
            elif risk_category == 'MEDIUM':
                recommendations = [
                    "⚠️ Moderate risk - Schedule routine follow-up",
                    "💊 Review medication adherence",
                    "📅 Consider preventive care measures",
                    "📈 Monitor for symptom progression"
                ]
                if risk_score > 60:
                    recommendations.append("🔄 Increase monitoring frequency")
            else:
                recommendations = [
                    "✅ Low risk - Continue routine care",
                    "🏃 Encourage healthy lifestyle maintenance",
                    "📅 Schedule annual wellness check",
                    "📚 Provide patient education resources"
                ]
            
            for rec in recommendations:
                st.write(f"• {rec}")
                
        except Exception as e:
            st.error(f"❌ Error calculating risk: {e}")

# Analytics section
st.header("📊 Analytics Dashboard")

# Fetch inference logs
try:
    logs_df = session.table("ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.INFERENCE_REQUEST_LOG").to_pandas()
    
    if not logs_df.empty:
        col1, col2 = st.columns(2)
        
        with col1:
            st.subheader("📈 Recent Inference Requests")
            st.dataframe(logs_df.tail(10))
        
        with col2:
            st.subheader("⚡ Response Time Distribution")
            if 'RESPONSE_TIME_MS' in logs_df.columns:
                fig_hist = px.histogram(
                    logs_df, 
                    x='RESPONSE_TIME_MS',
                    title="Response Time Distribution (ms)",
                    nbins=20
                )
                st.plotly_chart(fig_hist, use_container_width=True)
    else:
        st.info("No inference requests logged yet. Submit a patient assessment to see analytics.")
        
except Exception as e:
    st.warning(f"Analytics data not available: {e}")

# Model evaluation results
st.header("🎯 Model Performance")

try:
    eval_df = session.table("ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.MODEL_EVALUATION_LOG").to_pandas()
    
    if not eval_df.empty:
        col1, col2 = st.columns(2)
        
        with col1:
            st.subheader("📊 Latest Model Metrics")
            latest_eval = eval_df.iloc[-1]
            st.metric("MAE", f"{latest_eval.get('MAE', 0):.3f}")
            st.metric("RMSE", f"{latest_eval.get('RMSE', 0):.3f}")
            st.metric("R²", f"{latest_eval.get('R2_SCORE', 0):.3f}")
        
        with col2:
            st.subheader("🔄 Cross-Validation Results")
            if 'CV_SCORE_MEAN' in eval_df.columns:
                fig_cv = px.line(
                    eval_df, 
                    y='CV_SCORE_MEAN',
                    title="Cross-Validation Score Over Time"
                )
                st.plotly_chart(fig_cv, use_container_width=True)
    else:
        st.info("No model evaluation data available yet.")
        
except Exception as e:
    st.warning(f"Model evaluation data not available: {e}")

# Footer
st.markdown("---")
st.markdown("**🏥 Healthcare ML Platform** | Powered by Snowflake ML & Streamlit")
'''

# Write the Streamlit app to a file
with open('healthcare_dashboard.py', 'w') as f:
    f.write(streamlit_app_code)

print("✅ Streamlit app created: healthcare_dashboard.py")
print("📝 To run the dashboard:")
print("   streamlit run healthcare_dashboard.py")
print("")
print("🎯 Dashboard Features:")
print("   • Real-time patient risk assessment")
print("   • Interactive risk gauge and visualizations")
print("   • Clinical recommendations based on risk score")
print("   • Analytics dashboard with inference logs")
print("   • Model performance monitoring")
print("   • Responsive design with sidebar controls")


🖥️ Creating Streamlit healthcare dashboard...
✅ Streamlit app created: healthcare_dashboard.py
📝 To run the dashboard:
   streamlit run healthcare_dashboard.py

🎯 Dashboard Features:
   • Real-time patient risk assessment
   • Interactive risk gauge and visualizations
   • Clinical recommendations based on risk score
   • Analytics dashboard with inference logs
   • Model performance monitoring
   • Responsive design with sidebar controls


In [20]:
# Batch Inference Pipeline
print("📊 Setting up batch inference pipeline...")

def run_batch_inference(table_name: str, batch_size: int = 1000) -> Dict[str, Any]:
    """
    Run batch inference on a table of patients
    """
    print(f"🔄 Running batch inference on {table_name}...")
    
    start_time = time.time()
    
    try:
        # Get patient data for batch processing
        patient_data = session.sql(f"""
            SELECT 
                PATIENT_ID,
                AGE,
                NUM_CONDITIONS,
                NUM_MEDICATIONS,
                NUM_CLAIMS
            FROM {table_name}
            LIMIT {batch_size}
        """).collect()
        
        if not patient_data:
            return {"success": False, "error": "No patient data found"}
        
        # Run batch inference using UDF
        batch_sql = f"""
            SELECT 
                PATIENT_ID,
                AGE,
                NUM_CONDITIONS,
                NUM_MEDICATIONS,
                NUM_CLAIMS,
                healthcare_risk_score_udf(AGE, NUM_CONDITIONS, NUM_MEDICATIONS, NUM_CLAIMS) as RISK_SCORE,
                CASE 
                    WHEN healthcare_risk_score_udf(AGE, NUM_CONDITIONS, NUM_MEDICATIONS, NUM_CLAIMS) < 30 THEN 'LOW'
                    WHEN healthcare_risk_score_udf(AGE, NUM_CONDITIONS, NUM_MEDICATIONS, NUM_CLAIMS) < 70 THEN 'MEDIUM'
                    ELSE 'HIGH'
                END as RISK_CATEGORY,
                CURRENT_TIMESTAMP() as INFERENCE_TIMESTAMP
            FROM {table_name}
            LIMIT {batch_size}
        """
        
        results_df = session.sql(batch_sql)
        
        # Save batch results
        results_df.write.mode("overwrite").save_as_table("ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.BATCH_INFERENCE_RESULTS")
        
        # Get summary statistics
        summary = session.sql("""
            SELECT 
                COUNT(*) as TOTAL_PATIENTS,
                AVG(RISK_SCORE) as AVG_RISK_SCORE,
                COUNT(CASE WHEN RISK_CATEGORY = 'HIGH' THEN 1 END) as HIGH_RISK_COUNT,
                COUNT(CASE WHEN RISK_CATEGORY = 'MEDIUM' THEN 1 END) as MEDIUM_RISK_COUNT,
                COUNT(CASE WHEN RISK_CATEGORY = 'LOW' THEN 1 END) as LOW_RISK_COUNT
            FROM ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.BATCH_INFERENCE_RESULTS
        """).collect()[0]
        
        processing_time = (time.time() - start_time) * 1000
        
        result = {
            "success": True,
            "total_patients": summary['TOTAL_PATIENTS'],
            "avg_risk_score": float(summary['AVG_RISK_SCORE']),
            "high_risk_count": summary['HIGH_RISK_COUNT'],
            "medium_risk_count": summary['MEDIUM_RISK_COUNT'],
            "low_risk_count": summary['LOW_RISK_COUNT'],
            "processing_time_ms": processing_time,
            "throughput_patients_per_sec": summary['TOTAL_PATIENTS'] / (processing_time / 1000)
        }
        
        print(f"✅ Batch inference completed:")
        print(f"   📊 Processed: {result['total_patients']} patients")
        print(f"   ⚡ Processing time: {result['processing_time_ms']:.2f}ms")
        print(f"   🚀 Throughput: {result['throughput_patients_per_sec']:.1f} patients/sec")
        print(f"   📈 Risk distribution: {result['high_risk_count']} HIGH | {result['medium_risk_count']} MEDIUM | {result['low_risk_count']} LOW")
        
        return result
        
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "processing_time_ms": (time.time() - start_time) * 1000
        }

# Create sample patient data for batch testing
print("📝 Creating sample patient data for batch inference...")

try:
    # Create sample data
    sample_data_sql = """
        CREATE OR REPLACE TABLE ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.SAMPLE_PATIENTS AS
        SELECT 
            'PAT_' || ROW_NUMBER() OVER (ORDER BY UNIFORM(1, 1000, RANDOM())) as PATIENT_ID,
            UNIFORM(25, 85, RANDOM()) as AGE,
            UNIFORM(1, 15, RANDOM()) as NUM_CONDITIONS,
            UNIFORM(1, 20, RANDOM()) as NUM_MEDICATIONS,
            UNIFORM(5, 50, RANDOM()) as NUM_CLAIMS
        FROM TABLE(GENERATOR(ROWCOUNT => 100))
    """
    
    session.sql(sample_data_sql).collect()
    print("✅ Sample patient data created (100 patients)")
    
    # Run batch inference on sample data
    batch_results = run_batch_inference("ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.SAMPLE_PATIENTS", 100)
    
    if batch_results["success"]:
        print("✅ Batch inference pipeline is operational")
    else:
        print(f"⚠️ Batch inference failed: {batch_results.get('error')}")
        
except Exception as e:
    print(f"⚠️ Sample data creation failed: {e}")
    print("💡 Batch inference will be available once patient data exists")


📊 Setting up batch inference pipeline...
📝 Creating sample patient data for batch inference...
✅ Sample patient data created (100 patients)
🔄 Running batch inference on ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.SAMPLE_PATIENTS...
✅ Batch inference completed:
   📊 Processed: 100 patients
   ⚡ Processing time: 2515.64ms
   🚀 Throughput: 39.8 patients/sec
   📈 Risk distribution: 81 HIGH | 19 MEDIUM | 0 LOW
✅ Batch inference pipeline is operational
